In [4]:
import pandas as pd
from tqdm import tqdm
import random

from embedding import OneHotEmbedder, Doc2VecEmbedder
from vectorstore import SimpleVectorDatabase
from models import TextDoc, Vector

# Text Document Similarity
Create a python program that will compute the text document similarity between different documents. Your implementation will take a list of documents as an input text corpus, and it will compute a dictionary of words for the given corpus. Later, when a new document (i.e, search document) is provided, your implementation should provide a list of documents that are similar to the given search document, in descending order of their similarity with the search document.

For computing similarity between any two documents in our question, you can use the following distance measures (optionally, you can also use any other measure as well).
1. dot product between the two vectors
2. distance norm (or Euclidean distance) between two vectors e.g. $|| u − v ||$

As part of answering the question, you can also compare and comment on which of the two methods (or any other measure if you have used some other measure) will perform better and what are the reasons for it.

---

This notebook guids trough the implementation and intended use of it. For additional details see the project report (pdf).


## Data

In [5]:
# Lets load the dataset. We use a small sample dataset from wikipedia here. But in the 
# `data_load` notebook, you can find a script to download a larger dataset. The larger
# dataset will give better results, but is more computationally expensive. To test this
# script, the small dataset is sufficient.

# The dataset consists of multiple wikipedia articles. Each row contains the title of the
# article, the text of the article as well as the url.
dataset = pd.read_csv("data/wiki_subset_mini.csv", sep=",")
dataset.head()

,id,url,title,text
0,31945,https://en.wikipedia.org/wiki/List%20of%20metr...,List of metro systems,This list of metro systems includes electrifie...
1,37600,https://en.wikipedia.org/wiki/Axis,Axis,Axis may refer to:\n\nMathematics\nAxis (mathe...
2,37945,https://en.wikipedia.org/wiki/Giovanni%20Paisi...,Giovanni Paisiello,Giovanni Paisiello (or Paesiello; 9 May 1740 –...
3,20807,https://en.wikipedia.org/wiki/Merlin,Merlin,"Merlin (, , ) is a mythical figure prominently..."
4,621,https://en.wikipedia.org/wiki/Amphibian,Amphibian,"Amphibians are ectothermic, tetrapod vertebrat..."


In [6]:
# remove some random articles for later testing
# draw random sample of 5 articles
random.seed(42)
testing_indices = random.sample(range(len(dataset)), 5)

test_texts = dataset.loc[testing_indices,:].reset_index(drop=True)
train_texts = dataset.loc[~dataset.index.isin(testing_indices),:].reset_index(drop=True)

## Embedding

In [7]:
# This parameter is used to set the vector dimension of the embedding algorithm and the 
# vector database. The higher the dimension, the more accurate the results will be, but
# the more computationally expensive the calculations will be. OpenAI's embedding models
# use a dimension of 1536 as a reference.
# We suggest to use a dimension of 200 for the smaller datasets. For the larger datasets,
# this parameter can be increased. The larger the vector dimension, the more accurate the
# the embeddings are, because more information can be embedded. However, this might also
# increase noise and is more computationally expensive.

VECTOR_DIM = 150

In [24]:
# In this step the embedding model is fitted. This means that the model is trained on the
# given dataset. The user can choose which embedding algorithm to use - this implementation
# is designed to be easily extendable to other embedding algorithms while keeping the user
# interface the same. Every embedding implementation needs to have the methods `fit` and
# `embed`. The `fit` method is used to train the model on the given dataset. The `embed`
# method is used to embed a given text into a vector.

# The `OneHotEmbedder` is a simple embedding algorithm that uses a one-hot encoding to create
# the embedding-vectors. It is trained on the dataset by creating a vocabulary of all words
# and then creating a very sparse vector for each text. Each vector signals which words are
# present in the text. There are two different embedding methods available: additive and
# ont-hot. The additive method is the default method, it counts the number of occurences of
# each word in the text. The one-hot method only signals if a word is present in the text or
# not, without counting. After creating the vector for one text, the vector has a length
# of the corpus, which is really huge. To reduce the dimensionality of the vector, PCA is
# applied to the vector. This reduces the dimensionality to the given `vector_dim`
# parameter.

# The `Doc2VecEmbedder` is a more complex embedding algorithm from the gensim library. It
# is more computationally expensive, but also more accurate. Details can be found in the 
# gensim documentation. We won't go into detail here, since this is out of scope of this
# excercise. (gensim documentation: https://radimrehurek.com/gensim/models/doc2vec.html)


# Choose the embedder model to use: OneHot or Doc2Vec currently available
embedder = OneHotEmbedder(vector_dim=VECTOR_DIM, embedding_method="additive")
#embedder = Doc2VecEmbedder(vector_dim=VECTOR_DIM)

# fit the embedder model: this could take a view minutes
embedder.fit(list(train_texts.text))

2024-01-11 15:51:00,036::onehotembedder.py[_fit()]::INFO::Fitting embedder. Preprocessing documents...
100%|██████████| 295/295 [00:13<00:00, 21.58it/s]
2024-01-11 15:51:13,828::onehotembedder.py[_fit()]::INFO::Corpus created with 65281 words.
2024-01-11 15:51:13,830::onehotembedder.py[_fit()]::INFO::Reducing vector dimensions: fitting PCA (n=150)


In [26]:
# In the next step we create a vector database. This database is used to store the texts
# we want to search in later. We fill the database with the texts, embedded as vectors.
# Again we designed this implementation to be easily extendable to other database
# implementations. The only requirement is that the database has specific methods like
# `sim_search` or `upsert` that takes a vector as an argument and updates or inserts a
# vector into the database. 

# The `SimpleVectorDatabase` is a simple implementation of a vector database. It stores
# the vectors in a simple dictionary. For each wikipedia article, we create a TexDoc from
# the text. This TextDoc is the embedded as a vector using the embedder model, fitted 
# earlier. We add some metadata like the actual text, the title and the source and finally
# upsert the vector into the database. This process might take some time, depending on the
# size of the dataset.


# create vector database instance
vecdb = SimpleVectorDatabase(vector_dim=VECTOR_DIM)

# create vectors and store them in the database
for i in tqdm(range(len(train_texts))):
    row = train_texts.iloc[i]
    doc = TextDoc(row.text)
    vec = Vector(
        embedding=embedder.embed(doc),
        data=TextDoc(doc),
        metadata={"type": "wikipedia", "src": row.url, "title": row.title})
    vecdb.upsert(vec)

100%|██████████| 295/295 [00:16<00:00, 18.04it/s]


In [29]:
# Now we fitted an embedding model and created a vector database. We can now try to find
# similar documents in the database. First we need to embedd the new text into a vector
# using the same embedding model.


# Let's take the last row of the dataset and query the database for similar vectors.
row = test_texts.iloc[3,:]
new_doc = TextDoc(row.text)
print(new_doc)

TextDoc(36454): Stephen Gary Wozniak (; born August 11, 1950), also kno ...


In [31]:
# Now we create a embedding vector from the TextDoc
new_vec = Vector(
    embedding=embedder.embed(new_doc),
    data=TextDoc(new_doc),
    metadata={"type": "wikipedia", "src": row.url, "title": row.title}
)
print(new_vec)

<Vector 0c333e22-49f0-42b6-b576-0fc1bc421361 (150) : TextDoc(36454): Stephen Gary Wozniak (; born August 11, 1950), also kno ...>


In [32]:
# Now we can query the database for similar vectors. Select the measure to use and the
# number of similar vectors to return. Again, this implementation is designed to be easily
# extendable to other similarity measures. Currently implemented measures are: cosine,
# euclidean and dot. 
# Each result is the vector found in the database and the corresponding similarity score.

similar_vectors = vecdb.sim_search(new_vec, measure="dot", k=2)
similar_vectors

[{'vector': <Vector d78f6097-7fe4-4285-beb4-a70771ae34e8 (150) : TextDoc(62194): Dolly Rebecca Parton (born January 19, 1946) is an Amer ...>,
  'score': 4111.125644690877},
 {'vector': <Vector 35293ef6-b248-4bc5-9644-010a91562f4e (150) : TextDoc(29752): A wearable computer, also known as a wearable or body-b ...>,
  'score': 4089.148729202497}]

In [33]:
# Lets look at the results. Our search vector is the wikipedia article for Stephen Gary
# Wozniak, the co-founder of Apple. The most similar vector in our database is the
# wikipedia article of Dolly Parton, the american singer. The second most similar vector
# is the article about a warable computer. This is a good result, since the article about
# Dolly Parton is also about a famous person and the article about the warable computer is
# also about a technical topic. The similarity measure is not perfect, but it is a good
# start. The results can be improved by using a larger dataset and a more complex embedding.


print(f"Vector:       [title: {new_vec.metadata['title']}] {new_vec.data}")
for i in range(len(similar_vectors)):
    print(f"Most similar: [title: {similar_vectors[i]['vector'].metadata['title']}] {similar_vectors[i]['vector'].data}")

Vector:       [title: Steve Wozniak] TextDoc(36454): Stephen Gary Wozniak (; born August 11, 1950), also kno ...
Most similar: [title: Dolly Parton] TextDoc(62194): Dolly Rebecca Parton (born January 19, 1946) is an Amer ...
Most similar: [title: Wearable computer] TextDoc(29752): A wearable computer, also known as a wearable or body-b ...


In [34]:
# Lets try using another measure: the cosine similarity. This measure is more robust to
# different vector lengths and is a good choice for text embeddings. The results are
# even better than before. We once again get the article about warable computers, but also
# the article about Activision, a video game company. This is a good result.


similar_vectors = vecdb.sim_search(new_vec, measure="cosine", k=2)

print(f"Vector:       [title: {new_vec.metadata['title']}] {new_vec.data}")
for i in range(len(similar_vectors)):
    print(f"Most similar: [title: {similar_vectors[i]['vector'].metadata['title']}] {similar_vectors[i]['vector'].data}")

Vector:       [title: Steve Wozniak] TextDoc(36454): Stephen Gary Wozniak (; born August 11, 1950), also kno ...
Most similar: [title: Wearable computer] TextDoc(29752): A wearable computer, also known as a wearable or body-b ...
Most similar: [title: Activision] TextDoc(31711): Activision Publishing, Inc. is an American video game p ...


In [35]:
# Lastly lets try the euclidean distance. This measure returns an article about Akito
# Morita, the co-founder of Sony. The second most similar article is about Microserfs, a
# novel about a group of computer programmers. This also seems to be a pretty good result.


similar_vectors = vecdb.sim_search(new_vec, measure="euclidean", k=2)

print(f"Vector:       [title: {new_vec.metadata['title']}] {new_vec.data}")
for i in range(len(similar_vectors)):
    print(f"Most similar: [title: {similar_vectors[i]['vector'].metadata['title']}] {similar_vectors[i]['vector'].data}")

Vector:       [title: Steve Wozniak] TextDoc(36454): Stephen Gary Wozniak (; born August 11, 1950), also kno ...
Most similar: [title: Akio Morita] TextDoc(7900): Akio Morita (January 26, 1921 – October 3, 1999) was a  ...
Most similar: [title: Microserfs] TextDoc(10410): Microserfs, published by HarperCollins in 1995, is an e ...
